In [ ]:
import worldcat_expanding
import csv


In [ ]:
with open('output/bookstore.csv') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    bookstore_isbns = [row[7].replace('-', '')
                       for row in spamreader
                       if 'ISBN' not in row[7]
                       and 'E BOOK' not in row[7]]
    
    

In [ ]:
for isbn in bookstore_isbns:
    print(isbn)
    isbns, marcrecords = worldcat_expanding.main(isbn)
    print(isbns)
    break

In [ ]:
import requests
import lxml.etree as ET

In [ ]:
i = '9780887276385'
url = 'http://xisbn.worldcat.org/webservices/xid/isbn/{}?method=getEditions&format=xml&ai=mike.waugh'.format(i)
response = requests.get(url)
tree = ET.fromstring(response.content)
for child in tree:
    print(child.text)

In [ ]:
other_isbns, other_marcs = worldcat_expanding.main('9780887276408')
for isbn in sorted(other_isbns):
    print(isbn)
